In [10]:
# Rigorous Focused Validation: Prime Uniformity & Binary Inversions
# Focus on fixing the identified weaknesses with maximum rigor

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from decimal import Decimal, getcontext
import pandas as pd
from collections import defaultdict, Counter
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Ultra-high precision
getcontext().prec = 100

class RigorousValidator:
    """
    Focused validation addressing the specific weaknesses identified
    """
    def __init__(self):
        # Target constants with ultra-high precision
        self.ALPHA_INV = Decimal('137.035999084')  # 1/α - theoretically motivated modulus
        self.PHI = Decimal('1.6180339887498948482045868343656381177203091798057628621354486227052604628189024497072072041893227938183')
        self.PI = Decimal('3.1415926535897932384626433832795028841971693993751058209749445923078164062862089986280348253421170679')
        self.E = Decimal('2.7182818284590452353602874713526624977572470936999595749669676277240766303535475945713821785251664274')

        # Critical constants for testing
        self.test_constants = {
            'alpha_inv': self.ALPHA_INV,
            'phi': self.PHI,
            'pi': self.PI,
            'e': self.E,
            'sqrt2': Decimal('1.4142135623730950488016887242096980785696718753769480731766797379907324784621070388503875343276415727'),
            'sqrt3': Decimal('1.7320508075688772935274463415058723669428052538103806280558069794519330169088000370811461867572485757')
        }

def generate_prime_sieve(limit=1500000):
    """Generate primes using optimized sieve"""
    sieve = [True] * limit
    sieve[0] = sieve[1] = False

    for i in range(2, int(limit**0.5) + 1):
        if sieve[i]:
            for j in range(i*i, limit, i):
                sieve[j] = False

    return [i for i, is_prime in enumerate(sieve) if is_prime]

def rigorous_prime_uniformity_test(n_primes=114000, target_modulus=137):
    """
    Rigorous prime uniformity test focusing on the specific theoretical claim
    Tests for "too uniform" (χ² deficit) rather than generic patterns
    """
    print(f"🔬 RIGOROUS PRIME UNIFORMITY TEST")
    print(f"Testing {n_primes} primes mod {target_modulus} for 'too uniform' distribution")
    print("=" * 70)

    # Generate primes
    primes = generate_prime_sieve()[:n_primes]
    print(f"Generated {len(primes)} primes (largest: {primes[-1]})")

    # Test the theoretically motivated modulus (137 ≈ 1/α)
    residues = [p % target_modulus for p in primes]

    # Remove unavoidable zero class (as in the paper)
    non_zero_residues = [r for r in residues if r != 0]
    available_classes = target_modulus - 1  # 136 classes for mod 137

    print(f"Non-zero residues: {len(non_zero_residues)}")
    print(f"Available residue classes: {available_classes}")

    # Count frequency in each residue class
    residue_counts = np.bincount(non_zero_residues, minlength=target_modulus)[1:]  # Exclude class 0

    # Expected frequency under perfect uniformity
    expected_freq = len(non_zero_residues) / available_classes

    # Chi-square test for uniformity
    chi2_stat = np.sum((residue_counts - expected_freq)**2 / expected_freq)
    degrees_freedom = available_classes - 1
    p_value = 1 - stats.chi2.cdf(chi2_stat, degrees_freedom)

    # Calculate deficit/excess in terms of sigma
    expected_chi2 = degrees_freedom
    chi2_std = np.sqrt(2 * degrees_freedom)
    sigma_deviation = (chi2_stat - expected_chi2) / chi2_std

    print(f"\n📊 UNIFORMITY ANALYSIS:")
    print(f"χ² statistic: {chi2_stat:.1f}")
    print(f"Degrees of freedom: {degrees_freedom}")
    print(f"Expected χ² under uniformity: {expected_chi2}")
    print(f"Observed - Expected: {chi2_stat - expected_chi2:.1f}")
    print(f"Standard deviations from expected: {sigma_deviation:.1f}σ")
    print(f"P-value: {p_value:.2e}")

    # Interpret the result
    if sigma_deviation < -3:  # Significant deficit (too uniform)
        print(f"✅ SIGNIFICANT UNIFORMITY EXCESS: {abs(sigma_deviation):.1f}σ deficit")
        print("   Prime residues are significantly MORE uniform than expected")
        significant_uniformity = True
    elif sigma_deviation > 3:  # Significant excess (too patterned)
        print(f"⚠️  SIGNIFICANT NON-UNIFORMITY: {sigma_deviation:.1f}σ excess")
        significant_uniformity = False
    else:
        print(f"❌ NO SIGNIFICANT DEVIATION: {abs(sigma_deviation):.1f}σ from expected")
        significant_uniformity = False

    # Additional analysis: gap distribution
    gaps = np.diff(sorted(non_zero_residues))
    gap_uniformity = stats.kstest(gaps, 'uniform')

    print(f"\n📈 GAP ANALYSIS:")
    print(f"Residue gap KS test: D = {gap_uniformity.statistic:.4f}, p = {gap_uniformity.pvalue:.2e}")

    # Control test: random moduli
    print(f"\n🎲 CONTROL TEST: Random moduli")
    control_moduli = [127, 131, 139, 149]  # Primes near 137
    control_results = []

    for mod in control_moduli:
        ctrl_residues = [p % mod for p in primes if p % mod != 0]
        ctrl_expected = len(ctrl_residues) / (mod - 1)
        ctrl_counts = np.bincount(ctrl_residues, minlength=mod)[1:]
        ctrl_chi2 = np.sum((ctrl_counts - ctrl_expected)**2 / ctrl_expected)
        ctrl_df = mod - 2
        ctrl_sigma = (ctrl_chi2 - ctrl_df) / np.sqrt(2 * ctrl_df)
        control_results.append(ctrl_sigma)
        print(f"   Mod {mod}: {ctrl_sigma:.1f}σ deviation")

    avg_control_deviation = np.mean(np.abs(control_results))
    print(f"   Average control deviation: ±{avg_control_deviation:.1f}σ")

    if significant_uniformity and abs(sigma_deviation) > 2 * avg_control_deviation:
        print(f"✅ RESULT IS DISTINCTIVE: Mod 137 shows {abs(sigma_deviation)/avg_control_deviation:.1f}× stronger signal than controls")

    return {
        'chi2_stat': chi2_stat,
        'p_value': p_value,
        'sigma_deviation': sigma_deviation,
        'significant_uniformity': significant_uniformity,
        'control_deviations': control_results,
        'residue_counts': residue_counts
    }

def binary_string_inversion_test(constants_dict):
    """
    Test binary string inversions: 0.binary_string ↔ 1 - 0.binary_string
    Critical for understanding binary world inversions
    """
    print(f"\n🔄 BINARY STRING INVERSION TEST")
    print("Testing: 0.binary → 1 - 0.binary inversions")
    print("=" * 70)

    results = {}

    for const_name, const_val in constants_dict.items():
        # Convert to float for calculations
        const_float = float(const_val)
        print(f"\n📊 Testing {const_name} = {const_float:.10f}")

        # Convert to binary string representation
        decimal_part = const_float - int(const_float)  # Get fractional part

        # Generate binary digits of the fractional part
        binary_digits = []
        temp = decimal_part
        for _ in range(64):  # 64 binary digits
            temp *= 2
            if temp >= 1:
                binary_digits.append(1)
                temp -= 1
            else:
                binary_digits.append(0)

        # Create binary fraction
        binary_fraction = sum(bit * (2 ** -(i+1)) for i, bit in enumerate(binary_digits))

        # Calculate inversion: 1 - binary_fraction
        inverted_fraction = 1.0 - binary_fraction

        # Test if inversion relates to other constants
        inversion_matches = []
        for other_name, other_val in constants_dict.items():
            if other_name != const_name:
                other_float = float(other_val)
                # Test various scaling relationships with the inversion
                scales = [1.0, 0.5, 2.0, 0.1, 10.0, 1.0/3.0, 3.0, 1.0/7.0, 7.0]
                for scale in scales:
                    scaled_inversion = inverted_fraction * scale
                    diff = abs(scaled_inversion - other_float)

                    if diff < 1e-6:  # Significant match
                        inversion_matches.append({
                            'target': other_name,
                            'scale': scale,
                            'difference': diff,
                            'match_value': scaled_inversion
                        })

        # Test self-similarity: does the constant show up in its own binary inversion?
        self_similarity = abs(inverted_fraction - const_float)

        results[const_name] = {
            'binary_fraction': binary_fraction,
            'inverted_fraction': inverted_fraction,
            'inversion_matches': inversion_matches,
            'self_similarity': self_similarity,
            'binary_digits': binary_digits[:20]  # First 20 binary digits
        }

        print(f"   Binary fraction: {binary_fraction:.10f}")
        print(f"   Inverted (1-x): {inverted_fraction:.10f}")
        print(f"   Self-similarity: {self_similarity:.2e}")
        print(f"   Inversion matches: {len(inversion_matches)}")

        if inversion_matches:
            for match in inversion_matches[:3]:  # Show top 3
                print(f"     → {match['target']} × {match['scale']}: diff = {match['difference']:.2e}")

    return results

def base10_explosion_pattern_test(constants_dict):
    """
    Test for base-10 'explosions' - same digits at different scales
    Focus on powers of 10 and fractional relationships
    """
    print(f"\n💥 BASE-10 EXPLOSION PATTERN TEST")
    print("Testing digit patterns across power-of-10 scaling")
    print("=" * 70)

    def extract_digit_signature(number, precision=50):
        """Extract normalized digit signature"""
        # Convert to float to avoid Decimal issues
        num_float = float(number) if hasattr(number, '__float__') else number
        str_num = f"{abs(num_float):.{precision}f}".replace('.', '').lstrip('0')
        return str_num[:precision] if len(str_num) >= 10 else ""

    def calculate_digit_overlap(sig1, sig2, min_length=4):
        """Calculate overlapping digit patterns"""
        if len(sig1) < min_length or len(sig2) < min_length:
            return 0, []

        overlaps = []
        for i in range(len(sig1) - min_length + 1):
            for j in range(len(sig2) - min_length + 1):
                for length in range(min_length, min(len(sig1) - i, len(sig2) - j) + 1):
                    pattern1 = sig1[i:i+length]
                    pattern2 = sig2[j:j+length]
                    if pattern1 == pattern2:
                        overlaps.append((pattern1, i, j, length))

        # Return longest overlap
        if overlaps:
            best_overlap = max(overlaps, key=lambda x: x[3])
            return best_overlap[3], [best_overlap]
        return 0, []

    explosion_results = {}

    for const_name, const_val in constants_dict.items():
        print(f"\n🔍 Testing {const_name}")

        explosions = []
        base_signature = extract_digit_signature(const_val)
        const_float = float(const_val)

        # Test power-of-10 scaling
        for power in range(-6, 7):
            if power == 0:
                continue

            # Use float arithmetic to avoid Decimal/float mixing
            scaled_val = const_float * (10.0 ** power)
            scaled_signature = extract_digit_signature(scaled_val)

            overlap_length, overlaps = calculate_digit_overlap(base_signature, scaled_signature)

            if overlap_length >= 6:  # Significant overlap
                explosions.append({
                    'type': 'power10',
                    'scale_factor': 10.0 ** power,
                    'power': power,
                    'overlap_length': overlap_length,
                    'overlaps': overlaps,
                    'scaled_value': scaled_val
                })

        # Test fractional scaling (your 1/2 example)
        fractions = [0.5, 2.0, 1.0/3.0, 3.0, 1.0/7.0, 7.0, 1.0/137.0, 137.0]
        for frac in fractions:
            scaled_val = const_float * frac
            scaled_signature = extract_digit_signature(scaled_val)

            overlap_length, overlaps = calculate_digit_overlap(base_signature, scaled_signature)

            if overlap_length >= 6:
                explosions.append({
                    'type': 'fractional',
                    'scale_factor': frac,
                    'overlap_length': overlap_length,
                    'overlaps': overlaps,
                    'scaled_value': scaled_val
                })

        explosion_results[const_name] = explosions

        print(f"   Base signature: {base_signature[:20]}...")
        print(f"   Explosions found: {len(explosions)}")

        for explosion in explosions[:3]:  # Show top 3
            print(f"     {explosion['type']} × {explosion['scale_factor']}: {explosion['overlap_length']}-digit overlap")

    return explosion_results

def comprehensive_inversion_validation():
    """
    Comprehensive validation focusing on inversions and rigorous testing
    """
    print("🚀 COMPREHENSIVE INVERSION & RIGOUR VALIDATION")
    print("Focus: Prime uniformity, binary inversions, base-10 explosions")
    print("=" * 80)

    validator = RigorousValidator()

    # Test 1: Rigorous prime uniformity (the main weakness to fix)
    print("\n" + "="*80)
    print("TEST 1: RIGOROUS PRIME UNIFORMITY")
    prime_results = rigorous_prime_uniformity_test(n_primes=114000, target_modulus=137)

    # Test 2: Binary string inversions
    print("\n" + "="*80)
    print("TEST 2: BINARY STRING INVERSIONS")
    binary_results = binary_string_inversion_test(validator.test_constants)

    # Test 3: Base-10 explosion patterns
    print("\n" + "="*80)
    print("TEST 3: BASE-10 EXPLOSION PATTERNS")
    explosion_results = base10_explosion_pattern_test(validator.test_constants)

    # Comprehensive assessment
    print("\n" + "="*80)
    print("📊 COMPREHENSIVE ASSESSMENT")
    print("="*80)

    evidence_score = 0
    max_score = 3

    # Score prime uniformity
    if prime_results['significant_uniformity'] and abs(prime_results['sigma_deviation']) > 3:
        print("✅ PRIME UNIFORMITY: Significant evidence (137 mod shows distinctive behavior)")
        evidence_score += 1
    else:
        print("❌ PRIME UNIFORMITY: No significant evidence")

    # Score binary inversions
    total_inversion_matches = sum(len(result['inversion_matches']) for result in binary_results.values())
    if total_inversion_matches > 5:
        print(f"✅ BINARY INVERSIONS: Significant evidence ({total_inversion_matches} matches found)")
        evidence_score += 1
    else:
        print(f"❌ BINARY INVERSIONS: Insufficient evidence ({total_inversion_matches} matches)")

    # Score base-10 explosions
    total_explosions = sum(len(explosions) for explosions in explosion_results.values())
    if total_explosions > 10:
        print(f"✅ BASE-10 EXPLOSIONS: Significant evidence ({total_explosions} patterns found)")
        evidence_score += 1
    else:
        print(f"❌ BASE-10 EXPLOSIONS: Insufficient evidence ({total_explosions} patterns)")

    print(f"\n🎯 OVERALL EVIDENCE STRENGTH: {evidence_score}/{max_score}")

    if evidence_score >= 2:
        print("🎉 STRONG EVIDENCE: Multiple independent tests validate the framework")
    elif evidence_score == 1:
        print("⚠️  MODERATE EVIDENCE: Some tests validate, needs further investigation")
    else:
        print("❌ INSUFFICIENT EVIDENCE: Framework needs major revision")

    # Create focused visualizations
    try:
        create_focused_visualizations(prime_results, binary_results, explosion_results)
    except Exception as e:
        print(f"\n⚠️  Visualization error: {e}")
        print("Continuing with text-based results...")

        # Fallback: print key results
        print(f"\n📊 TEXT-BASED RESULTS SUMMARY:")
        print(f"Prime uniformity: χ² = {prime_results['chi2_stat']:.1f}, σ = {prime_results['sigma_deviation']:.1f}")

        total_binary_matches = sum(len(result['inversion_matches']) for result in binary_results.values())
        print(f"Binary inversion matches: {total_binary_matches}")

        total_explosions = sum(len(explosions) for explosions in explosion_results.values())
        print(f"Base-10 explosion patterns: {total_explosions}")

    return {
        'prime_results': prime_results,
        'binary_results': binary_results,
        'explosion_results': explosion_results,
        'evidence_score': evidence_score
    }

def create_focused_visualizations(prime_results, binary_results, explosion_results):
    """Create focused visualizations for the key tests"""

    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Focused Validation Results: Rigorous Testing', fontsize=16, fontweight='bold')

    # Plot 1: Prime residue uniformity
    ax1 = axes[0, 0]
    residue_counts = prime_results['residue_counts']
    expected_freq = float(np.mean(residue_counts))

    x = list(range(1, len(residue_counts) + 1))
    ax1.bar(x, residue_counts, alpha=0.7, color='skyblue', label='Observed')
    ax1.axhline(expected_freq, color='red', linestyle='--', linewidth=2, label=f'Expected ({expected_freq:.1f})')

    ax1.set_xlabel('Residue Class (mod 137)')
    ax1.set_ylabel('Frequency')
    ax1.set_title(f'Prime Residue Distribution\nχ² = {prime_results["chi2_stat"]:.1f}, {prime_results["sigma_deviation"]:.1f}σ deviation')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Plot 2: Binary inversion matches
    ax2 = axes[0, 1]
    const_names = list(binary_results.keys())
    match_counts = [len(result['inversion_matches']) for result in binary_results.values()]

    bars = ax2.bar(const_names, match_counts, alpha=0.7, color='lightcoral')
    ax2.set_xlabel('Constants')
    ax2.set_ylabel('Number of Inversion Matches')
    ax2.set_title('Binary String Inversion Matches')
    ax2.tick_params(axis='x', rotation=45)

    # Add value labels on bars
    for bar, count in zip(bars, match_counts):
        if count > 0:
            ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                    str(count), ha='center', va='bottom')

    # Plot 3: Base-10 explosion patterns
    ax3 = axes[1, 0]
    explosion_counts = [len(explosions) for explosions in explosion_results.values()]

    # Ensure all data is properly converted to basic Python types
    explosion_counts = [int(count) for count in explosion_counts]
    const_names_clean = [str(name) for name in const_names]

    bars = ax3.bar(const_names_clean, explosion_counts, alpha=0.7, color='lightgreen')
    ax3.set_xlabel('Constants')
    ax3.set_ylabel('Number of Explosion Patterns')
    ax3.set_title('Base-10 Explosion Pattern Detection')
    ax3.tick_params(axis='x', rotation=45)

    # Add value labels
    for bar, count in zip(bars, explosion_counts):
        if count > 0:
            ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                    str(count), ha='center', va='bottom')

    # Plot 4: Overall evidence summary
    ax4 = axes[1, 1]

    tests = ['Prime\nUniformity', 'Binary\nInversions', 'Base-10\nExplosions']
    scores = []

    # Prime uniformity score
    if prime_results['significant_uniformity'] and abs(prime_results['sigma_deviation']) > 3:
        scores.append(1)
    else:
        scores.append(0)

    # Binary inversion score
    total_matches = sum(len(result['inversion_matches']) for result in binary_results.values())
    scores.append(1 if total_matches > 5 else 0)

    # Explosion score
    total_explosions = sum(len(explosions) for explosions in explosion_results.values())
    scores.append(1 if total_explosions > 10 else 0)

    colors = ['green' if score == 1 else 'red' for score in scores]
    bars = ax4.bar(tests, scores, alpha=0.7, color=colors)
    ax4.set_ylabel('Evidence Score (1=Strong, 0=Weak)')
    ax4.set_title('Overall Evidence Assessment')
    ax4.set_ylim(0, 1.2)

    # Add labels
    for bar, score in zip(bars, scores):
        label = 'STRONG' if score == 1 else 'WEAK'
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
                label, ha='center', va='bottom', fontweight='bold')

    plt.tight_layout()
    plt.show()

    return fig

# Execute the comprehensive validation
if __name__ == "__main__":
    print("🔬 RIGOROUS FOCUSED VALIDATION")
    print("Addressing identified weaknesses with maximum scientific rigor")
    print("Focus: Prime uniformity deficit, binary inversions, base-10 scaling")
    print()

    results = comprehensive_inversion_validation()

    print(f"\n📋 VALIDATION SUMMARY:")
    print(f"This rigorous analysis specifically addresses the weaknesses identified")
    print(f"in the previous statistical validation, focusing on:")
    print(f"1. Prime residue analysis with proper theoretical motivation")
    print(f"2. Binary string inversion relationships")
    print(f"3. Base-10 scaling 'explosion' patterns")
    print(f"4. Maximum statistical rigor and proper controls")

    evidence_score = results['evidence_score']
    if evidence_score >= 2:
        print(f"\n🏆 CONCLUSION: Strong evidence ({evidence_score}/3 tests passed)")
        print(f"Multiple independent validations support the theoretical framework")
    else:
        print(f"\n⚠️  CONCLUSION: Needs further work ({evidence_score}/3 tests passed)")
        print(f"Some aspects validated, but framework requires refinement")

🔬 RIGOROUS FOCUSED VALIDATION
Addressing identified weaknesses with maximum scientific rigor
Focus: Prime uniformity deficit, binary inversions, base-10 scaling

🚀 COMPREHENSIVE INVERSION & RIGOUR VALIDATION
Focus: Prime uniformity, binary inversions, base-10 explosions

TEST 1: RIGOROUS PRIME UNIFORMITY
🔬 RIGOROUS PRIME UNIFORMITY TEST
Testing 114000 primes mod 137 for 'too uniform' distribution
Generated 114000 primes (largest: 1497541)
Non-zero residues: 113999
Available residue classes: 136

📊 UNIFORMITY ANALYSIS:
χ² statistic: 24.1
Degrees of freedom: 135
Expected χ² under uniformity: 135
Observed - Expected: -110.9
Standard deviations from expected: -6.7σ
P-value: 1.00e+00
✅ SIGNIFICANT UNIFORMITY EXCESS: 6.7σ deficit
   Prime residues are significantly MORE uniform than expected

📈 GAP ANALYSIS:
Residue gap KS test: D = 0.9988, p = 0.00e+00

🎲 CONTROL TEST: Random moduli
   Mod 127: -6.4σ deviation
   Mod 131: -6.5σ deviation
   Mod 139: -6.8σ deviation
   Mod 149: -6.8σ deviati